In [9]:
import os
os.environ['OPENBLAS_NUM_THREADS'] = '1'   
import pathlib
import importlib
import sys
import warnings
from functools import partial as bind


import embodied
from embodied import wrappers
import dreamerv3

from sogym.env_gym import sogym
import warnings
import dreamerv3
from dreamerv3 import agent as agt

from embodied.envs import from_gym

In [54]:

warnings.filterwarnings('ignore', '.*truncated to dtype int32.*')

# See configs.yaml for all options.
config = embodied.Config(agt.Agent.configs['defaults'])
config = config.update(agt.Agent.configs['size12m'])
  


from embodied.envs import from_gym

env = sogym(mode='train',observation_type='topopt_game',vol_constraint_type = 'hard',resolution=50,img_format = 'HWC',check_connectivity=True) # Replace this with your Gym env.
# env = StepAPICompatibility(env[0])
env = from_gym.FromGym(env)


agent = agt.Agent(env.obs_space, env.act_space, config)
eval_policy = lambda *args: agent.policy(*args, mode='eval')

checkpoint = embodied.Checkpoint()
checkpoint.agent = agent
checkpoint.load('/home/thomas/datacortex/log_dreamer/logdir/12m_topopt_nostrain/checkpoint.ckpt', keys=['agent'])

Observation space
  beta             Space(dtype=float32, shape=(27,), low=-1.0, high=2.0)
  design_variables Space(dtype=float32, shape=(48,), low=-1.0, high=1.0)
  image            Space(dtype=uint8, shape=(64, 64, 3), low=0, high=255)
  n_steps_left     Space(dtype=float32, shape=(1,), low=0.0, high=1.0)
  score            Space(dtype=float32, shape=(1,), low=-inf, high=inf)
  structure_strain_energy Space(dtype=uint8, shape=(64, 64, 3), low=0, high=255)
  volume           Space(dtype=float32, shape=(1,), low=0.0, high=1.0)
  reward           Space(dtype=float32, shape=(), low=-inf, high=inf)
  is_first         Space(dtype=bool, shape=(), low=False, high=True)
  is_last          Space(dtype=bool, shape=(), low=False, high=True)
  is_terminal      Space(dtype=bool, shape=(), low=False, high=True)
Action space
  action           Space(dtype=float32, shape=(6,), low=-1.0, high=1.0)
  reset            Space(dtype=bool, shape=(), low=False, high=True)
Encoder: {'beta': (27,), 'design_var

In [89]:
def make_eval_env(config, env_id=0):
    from embodied.envs import from_gym
    from sogym.env_gym import sogym
    env = sogym(mode='test',observation_type='topopt_game',vol_constraint_type = 'hard',resolution=50,img_format = 'HWC',check_connectivity=True,use_std_strain=False) # Replace this with your Gym env.
    env = from_gym.FromGym(env)
    return env

fns = [bind(make_eval_env, i) for i in range(1)]
eval_driver = embodied.Driver(fns, False)
eval_driver.on_step(lambda tran, _: step.increment())


In [192]:
eval_driver.reset(agent.init_policy)

In [201]:
eval_driver(eval_policy, steps=1)
print(eval_driver.envs[0]._env.Phi.shape)
fig = eval_driver.envs[0]._env.plot(train_viz=False)
fig.savefig('./test.png')

(6536, 8)


In [202]:
print(eval_driver.envs[0]._env.compliance)

predicted_volume = eval_driver.envs[0]._env.volume
desired_volume = eval_driver.envs[0]._env.conditions['volfrac']

print((predicted_volume-desired_volume)/desired_volume)

74.14160186457302
-0.16469878839385527


In [101]:
print(predicted_volume)

0.34334863982390557


In [205]:
import numpy as np
logged_comp = [
    55.41,
    37.2,
    51.55,
    48.24,
    12.75,
    29.68,
    232.63,
    37.63,
    267.36,
    74.14,
]

log_deltavol =[
13.15,
28.1,
13.97,
36.43,
8.37,
30.63,
26.97,
29.84,
28.26,
16.47
]

print(np.median(np.array(logged_comp)))
print(np.mean(np.array(log_deltavol)))

49.894999999999996
23.219
